In [ ]:
%matplotlib widget

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import flammkuchen as fl

from glob import glob
import pandas as pd
from bouter import Experiment
import colorspacious
import json
from scipy import stats

In [ ]:
master_pre = Path(r"Z:\Hagar\E0040\v31\pre ablation")
pre_list = list(master_pre.glob("*_f*"))
pre_list = pre_list[1:3]+[pre_list[4]]

master_post1 = Path(r"Z:\Hagar\E0040\v31\post ablation 1d")
post1_list = list(master_post1.glob("*_f*"))
post1_list = [post1_list[1]] + post1_list[3:5]

master_post2 = Path(r"Z:\Hagar\E0040\v31\post ablation 2d")
post2_list = list(master_post2.glob("*_f*"))
post2_list = post2_list[1:3]+[post2_list[4]]

num_fish = len(pre_list)

In [ ]:
post1_list

In [ ]:
regions = ['r_habenula', 'l_habenula', 'ahb', 'ipn']
num_regions = len(regions)
thresh = 0.5
n_tuned_pre = np.zeros((num_regions, num_fish)) # habenula, aHB, IPN
n_neurons_pre = np.zeros((num_regions, num_fish)) # habenula, aHB, IPN

n_tuned_post1 = np.zeros((num_regions, num_fish)) # habenula, aHB, IPN
n_neurons_post1 = np.zeros((num_regions, num_fish)) # habenula, aHB, IPN

n_tuned_post2 = np.zeros((num_regions, num_fish)) # habenula, aHB, IPN
n_neurons_post2 = np.zeros((num_regions, num_fish)) # habenula, aHB, IPN

In [ ]:
bins = np.linspace(-1, 1, 100)    
fig2, ax2 = plt.subplots(num_regions, num_fish, figsize=(12, 7))

for fish in range(num_fish):
    pre_fish = pre_list[fish]
    exp = Experiment(pre_fish)
    fish_id = exp.fish_id
    print(fish_id)
    
    traces = fl.load(pre_fish / "filtered_traces.h5", "/detr")
    sensory_regressors = fl.load(pre_fish / "sensory_regressors.h5", "/regressors")
    coords_regions = fl.load(pre_fish / "registration" / "ref_brain_regions.h5")
    
    for region in range(num_regions):
        print(regions[region])
        region_idx = coords_regions[regions[region]]
        num_traces = np.shape(region_idx)[0]
        traces_region = traces[:, region_idx]

        reg_corr = np.zeros((2, num_traces))

        count=0
        for direction in [0, 4]:
            current_dir = np.asarray(sensory_regressors.iloc[:, direction])

            for i in range(num_traces):
                reg_corr[count, i] = np.corrcoef(current_dir, traces_region[:, i])[0,1]
            count += 1
        
        n_neurons_pre[region, fish] = num_traces
        n_tuned_pre[region, fish] = len(np.where(np.abs(reg_corr) > thresh)[0])
        ax2[region, fish].hist(np.append(reg_corr[0], reg_corr[1]), bins, alpha=0.5, label='pre', density=False)
        ax2[region, 0].set_ylabel("Count(" + regions[region] + ")")
    
    ax2[0, fish].set_title(fish_id)
    ax2[0, fish].set_xlabel("Correlation")
    
    ####################################### post1
    post_fish = post1_list[fish]
    
    traces = fl.load(post_fish / "filtered_traces.h5", "/detr")
    sensory_regressors = fl.load(post_fish / "sensory_regressors.h5", "/regressors")
    coords_regions = fl.load(post_fish / "registration" / "ref_brain_regions.h5")
        
    for region in range(num_regions):
        region_idx = coords_regions[regions[region]]
        num_traces = np.shape(region_idx)[0]
        traces_region = traces[:, region_idx]
        
        reg_corr = np.zeros((2, num_traces))
        count=0
        for direction in [0, 4]:
            current_dir = np.asarray(sensory_regressors.iloc[:, direction])

            for i in range(num_traces):
                reg_corr[count, i] = np.corrcoef(current_dir, traces_region[:, i])[0,1]
            count += 1

        n_neurons_post1[region, fish] = num_traces
        n_tuned_post1[region, fish] = len(np.where(np.abs(reg_corr) > thresh)[0])
        ax2[region, fish].hist(np.append(reg_corr[0], reg_corr[1]), bins, alpha=0.5, label='post', density=False)
        ax2[0, num_fish-1].legend(loc='upper right')
        
        ax2[region, fish].spines['right'].set_visible(False)
        ax2[region, fish].spines['top'].set_visible(False)
        
        
    ####################################### post2
    post_fish = post2_list[fish]
    
    traces = fl.load(post_fish / "filtered_traces.h5", "/detr")
    sensory_regressors = fl.load(post_fish / "sensory_regressors.h5", "/regressors")
    coords_regions = fl.load(post_fish / "registration" / "ref_brain_regions.h5")
        
    for region in range(num_regions):
        region_idx = coords_regions[regions[region]]
        num_traces = np.shape(region_idx)[0]
        traces_region = traces[:, region_idx]
        
        reg_corr = np.zeros((2, num_traces))
        count=0
        for direction in [0, 4]:
            current_dir = np.asarray(sensory_regressors.iloc[:, direction])

            for i in range(num_traces):
                reg_corr[count, i] = np.corrcoef(current_dir, traces_region[:, i])[0,1]
            count += 1

        n_neurons_post2[region, fish] = num_traces
        n_tuned_post2[region, fish] = len(np.where(np.abs(reg_corr) > thresh)[0])
        ax2[region, fish].hist(np.append(reg_corr[0], reg_corr[1]), bins, alpha=0.5, label='post', density=False, color='orchid')
        ax2[0, num_fish-1].legend(loc='upper right')
        
        ax2[region, fish].spines['right'].set_visible(False)
        ax2[region, fish].spines['top'].set_visible(False)
   

In [ ]:
file_name = "corr histogram pre and post norm 3 days.jpg"
fig2.savefig(master_pre / file_name, dpi=300)
file_name = "corr histogram pre and post norm 3 days.pdf"
fig2.savefig(master_pre / file_name, dpi=300)

In [ ]:
titles = ['Left habenula', 'Right habenula', 'aHB', 'IPN']

In [ ]:
fig3, ax3 = plt.subplots(2, num_regions, figsize=(8, 5))
ylim=[[0.2, 0.1, 1, 0.4], [1,1,1,1]]
for j in range(2):
    for i in range(num_regions):
        curr_plot = np.zeros((3, num_fish))
        curr_plot[0] = n_tuned_pre[i] / n_neurons_pre[i] / 2
        curr_plot[1] = n_tuned_post1[i] / n_neurons_post1[i] / 2
        curr_plot[2] = n_tuned_post2[i] / n_neurons_post2[i] / 2
        ax3[j, i].plot(curr_plot, c='skyblue')
        x = np.mean(curr_plot, axis=1)
        err = np.std(curr_plot) / num_fish
        ax3[j, i].errorbar([0, 1, 2], x, yerr=err, c='royalblue')


        ax3[0, i].set_title(titles[i])
        ax3[j, i].spines['right'].set_visible(False)
        ax3[j, i].spines['top'].set_visible(False)

        labels = [item.get_text() for item in ax3[j, i].get_xticklabels()]
        labels[1] = 'Pre'
        labels[2] = 'Post 1d'
        labels[3] = 'Post 2d'

        ax3[j, i].set_xticklabels(labels)
        pval_01 = round(stats.ttest_rel(curr_plot[0], curr_plot[1])[1], 3)
        pval_02 = round(stats.ttest_rel(curr_plot[0], curr_plot[2])[1], 3)
        pval_12 = round(stats.ttest_rel(curr_plot[1], curr_plot[2])[1], 3)
        ax3[j, i].set_ylim(0, ylim[j][i])
        ax3[0, i].text(0.1, ylim[j][i]*0.75, str(pval_01), fontsize=8)
        ax3[0, i].text(0.75, ylim[j][i]*0.9, str(pval_02), fontsize=8)
        ax3[0, i].text(1.2, ylim[j][i]*0.75, str(pval_12), fontsize=8)


    ax3[j, 0].set_ylabel('Tuned')
fig3.subplots_adjust(wspace=0.5, hspace=0.5)

In [ ]:
file_name = "precent change in tuned neurons 3 days" + str(thresh) + ".jpg"
fig3.savefig(master_pre / file_name, dpi=300)
file_name = "precent change in tuned neurons 3 days" + str(thresh) + ".pdf"
fig3.savefig(master_pre / file_name, dpi=300)